In [ ]:
import os
len(os.listdir('/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/train'))

566

In [ ]:
minutes1 = open('/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/train/task_B_en_train_5537/task_B_en_train_5537_minutes.txt', 'r')
minutes1 = minutes1.readlines()
print(len(minutes1))

31


In [ ]:
transcripts1 = open('/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/train/task_B_en_train_5537/task_B_en_train_5537_transcript.txt', 'r')
transcripts1 = transcripts1.readlines()
print(len(transcripts1))

148


In [ ]:
import re
 
def rem_boundries(wordList):
 
  processed_list = []
 
  reg_Ex = [r"\(", r"\)", r"\[", r"]"]
 
  for word in wordList:
    for exp in reg_Ex:
      word = re.sub(exp, '', word)
    processed_list.append(word)
 
  return processed_list

In [ ]:
def brac_match(reg_ex, TextList):
  
  processed_list = []
 
  for line in TextList:
    for exp in reg_ex:
      result = re.findall(exp, line)
      if len(result) > 0:
        for word in result:
          if word not in processed_list:
            processed_list.append(word)
 
  return processed_list

In [ ]:
import re 
 
def get_brac_cnt(Textlist):
  
  Reg_Ex = [r"\(PERSON\d+\)", r"\[PERSON\d+\]"] 
  Reg_Ex_ = [r"\[ORGANIZATION\d+\]", r"\[PROJECT\d+\]"]
  
  List_A = brac_match(Reg_Ex, Textlist)
  List_A =  rem_boundries(List_A)
 
  List_B = brac_match(Reg_Ex_, Textlist)
  List_B =  rem_boundries(List_B)
 
  if len(List_B) > len(List_A):
    return List_B
 
  return list(set(List_A))
 
# x, y = get_brac_cnt(minutes), get_brac_cnt(transcripts)

In [ ]:
def chk_person_match(meet_br_cnt, trans_br_cnt):
 
  similar_cnt = []
 
  for word in meet_br_cnt:
    if word in trans_br_cnt:
      similar_cnt.append(word)
 
  return similar_cnt
 
# c = chk_person_match(x, y)
# c, len(c), len(x)

In [ ]:
import re
 
def remove_cntsqbr(text_list):
  
  new_list = []
 
  for word in text_list:
    new_list.append(re.sub(r"( +)\[(.*?)\]", '', word))
 
  return new_list

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
 
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from nltk.tokenize import RegexpTokenizer
 
def remove_punctuation(text):
 
  tokenizer = RegexpTokenizer(r"\w+")
 
  return ' '.join(tokenizer.tokenize(text))

In [ ]:
def remove_stopwords(stop_words, text):
 
  wordsList = word_tokenize(text)
 
  wordsList = [w for w in wordsList if w not in stop_words]
 
  return wordsList

In [ ]:
def add_postag(word_tokens):
  
  tagged = nltk.pos_tag(word_tokens)
 
  return tagged

In [ ]:
text = "I got model description from [PERSON16]. Still waiting for training reproduction scripts for easy training setup"
text = remove_punctuation(text)
word_tokens = remove_stopwords(stop_words, text)
tag_list = add_postag(word_tokens)
nouns = get_NNS(tag_list)
print(nouns)

['model', 'description', 'reproduction', 'easy', 'setup']


In [ ]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import spacy

def add_postag_spacy(text):

  tagged = []

  nlp = spacy.load("en_core_web_sm")

  word_tokens = nlp(text)

  for token in word_tokens:
    tagged.append((token, token.pos_))

  return tagged

In [ ]:
def get_NOUNS_spacy(tagged_tokens):

  Extracted_Nouns = []

  Extracted_Nouns = [word for word, tag in tagged_tokens if tag == 'NOUN']

  return Extracted_Nouns

# get_NOUNS_spacy(tagged)

In [ ]:
def aggregate_nouns_spacy(textList):

  tot_nouns = []

  for text in textList:
    text = remove_punctuation(text)
    tag_list = add_postag_spacy(text)
    nouns = get_NOUNS_spacy(tag_list)
    tot_nouns.extend(nouns)

  tot_nouns = [word for word in tot_nouns if len(word) > 6]

  return tot_nouns

In [ ]:
def get_NNS(tagged_tokens):
 
  Extracted_Nouns = []
    
  # Extracted_Nouns = [word for word, tag in tagged_tokens if tag == 'NN' or tag == 'NNS']
  Extracted_Nouns = [word for word, tag in tagged_tokens if tag == 'NN' or tag == 'JJ']
  # Extracted_Nouns = [word for word, tag in tagged_tokens if tag == 'NNP' or tag == "NN"]
 
  return Extracted_Nouns

In [ ]:
def aggregate_nouns(stop_words, textList):
 
  tot_nouns = []
 
  for text in textList:
    text = remove_punctuation(text)
    word_tokens = remove_stopwords(stop_words, text)
    tag_list = add_postag(word_tokens)
    nouns = get_NNS(tag_list)
    tot_nouns.extend(nouns)
 
  tot_nouns = [word for word in tot_nouns if len(word) > 5]
 
  return tot_nouns
 
x = aggregate_nouns(stop_words, transcripts)
z = aggregate_nouns(stop_words, minutes)

NameError: ignored

In [ ]:
from collections import Counter

In [ ]:
def Counter_Transcripts(transcript):
 
  return Counter(transcript)

In [ ]:
def restrict_counter(Counter_Transcript):
 
    prepro_counter = dict()
 
    for key, value in Counter_Transcript.items():
 
      # if value < 10:
      #   prepro_counter[key] = value
 
      if value > 10:
        prepro_counter[key] = 10
      else:
        prepro_counter[key] = value
 
 
    return prepro_counter

In [ ]:
def get_similar_nouns(minute_nouns, transcript_nouns):
 
  similar_nouns = [w for w in minute_nouns if w in transcript_nouns]
 
  return similar_nouns

In [ ]:
def meet_word_count(minute_nouns, transcript_nouns_Counter):
 
  transcript_Counter = restrict_counter(transcript_nouns_Counter)
 
  trans_nouns_set = list(transcript_Counter.keys())
  trans_nouns_count = list(transcript_Counter.values())
 
  count_p_index = []
 
  for word in minute_nouns:
    for idx, t_word in enumerate(trans_nouns_set):
      if word == t_word:
        count_p_index.append(trans_nouns_count[idx])
 
  return count_p_index

In [ ]:
# Remove the content within square brackets 
minute = remove_cntsqbr(minutes)
transcript = remove_cntsqbr(transcripts)  
 
# Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
Minute_nouns = aggregate_nouns(stop_words, minute)
Transcript_nouns = aggregate_nouns(stop_words, transcript)
 
# Apply Counter to Transcripts
Transcript_nouns_Counter = Counter(Transcript_nouns)
 
# check the word count corresponding to similarity of minute nouns in transcript nouns
word_count = meet_word_count(Minute_nouns, Transcript_nouns_Counter)
print(word_count)
 
print(calculate_Avg(word_count))

[4, 1, 1, 3, 3, 8, 4, 2]
3.25


In [ ]:
import torch
 
def calculate_Avg(minute_Count):
 
  c_Tensor = torch.Tensor(minute_Count)
 
  mean = torch.mean(c_Tensor)
 
  return mean.item()

In [ ]:
def concat_nouns(minute_list, transcript_list):
 
  Nounlist_A = list(set(minute_list))
 
  trans_Counter = Counter_Transcripts(transcript_list)
  Trans_dict = restrict_counter(trans_Counter)  
  Nounlist_B = Trans_dict.keys()
 
  minute_str = ' '.join(Nounlist_A)
  transcript_str = ' '.join(Nounlist_B)
 
  return minute_str, transcript_str

In [ ]:
# Remove the content within square brackets 
minute = remove_cntsqbr(minutes)
transcript = remove_cntsqbr(transcripts)  
 
# Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
Minute_nouns = aggregate_nouns(stop_words, minute)
Transcript_nouns = aggregate_nouns(stop_words, transcript)
 
minute_str, transcript_str = concat_nouns(Minute_nouns, Transcript_nouns)

## Generate Label and File Lists

In [ ]:
cd /content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en

/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en


In [ ]:
import os

root_dir = '/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/'
train_dir = os.path.join(root_dir, 'train/')
dev_dir = os.path.join(root_dir, 'dev/')

In [ ]:
import pandas as pd 

train_tsv = pd.read_csv('/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/EN_train_task_b.tsv', sep='\t')

dir = list(train_tsv['Directory'])
labels = list(train_tsv['Label'])

In [ ]:
true_labels = [label for label in labels if label == True]
print(len(true_labels))

87


In [ ]:
print(len(os.listdir(train_dir)) + len(os.listdir(dev_dir)))
print(len(os.listdir(train_dir)))
print(len(os.listdir(dev_dir)))

846
566
280


In [ ]:
import pandas as pd 

dev_tsv = pd.read_csv('/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/EN_dev_task_b.tsv', sep='\t')

d_dir = list(dev_tsv['Directory'])
d_labels = list(dev_tsv['Label'])

In [ ]:
d_true_labels = [label for label in d_labels if label == True]
print(len(d_true_labels))

28


In [ ]:
def return_label(key, labels, dir):

  label = None
  for idx, i in enumerate(dir):
    if i == key:
      label = labels[idx]
  return label

In [ ]:
transcript_files = []
minutes_files = []
 
train_folder = os.listdir(train_dir)
 
for folder in train_folder:
 
  root_file = train_dir + f'{folder}'
  try:
    t_file = root_file + f'/{folder}_transcript.txt'
    open(t_file, 'r').readlines()
    m_file = root_file + f'/{folder}_minutes.txt'
    open(m_file, 'r').readlines()
 
    transcript_files.append(t_file)
    minutes_files.append(m_file)
 
  except Exception as e:
    print(f'Cannot open file in folder: {folder}')

In [ ]:
dev_folder = os.listdir(dev_dir)

for folder in dev_folder:

  root_file = dev_dir + f'{folder}'
  try:
    t_file = root_file + f'/{folder}_transcript.txt'
    open(t_file, 'r').readlines()
    m_file = root_file + f'/{folder}_minutes.txt'
    open(m_file, 'r').readlines()

    transcript_files.append(t_file)
    minutes_files.append(m_file)

  except Exception as e:
    print(f'Cannot open file in folder: {folder}')

In [ ]:
len(transcript_files)

In [ ]:
# transit from train->dev set
import re

def flag_train_2_dev(text):
  
  result = re.findall(r'dev', text)
  if len(result) > 0:
    return True
  else:
    return False

In [ ]:
Labels = []

for file_names in transcript_files:

  Flag = flag_train_2_dev(file_names)

  if Flag:
    print('Parsing dev files')
    key = file_names[-19: -15]
    print(key)
    try:
      key = int(key)
      Labels.append(return_label(key, d_labels, d_dir))
    except Exception as e:
      print(f'Cannot convert key in file {file_names} to int')

  else:
    print('Parsing train files')
    key = file_names[-19: -15]
    try:
      key = int(key)
      Labels.append(return_label(key, labels, dir))
    except Exception as e:
      print(f'Cannot convert key in file {file_names} to int')

## Calculate Score 1 for True Labels

In [ ]:
score_1_results = []

for idx, val in enumerate(Labels):
  print(f'Iterating over file: {idx}')
  try:
    minutes = open(minutes_files[idx], 'r').readlines()
    transcript = open(transcript_files[idx], 'r').readlines()
    pred_label = cnt_aggregator(minutes, transcript)
    score_1_results.append(pred_label)
  except Exception as e:
    print('Ignoring this files')

## Calculate Score 1

### Testing something related to Score 1

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
def lowercase(word_list):

  lower_list = []

  for word in word_list:
    lower_list.append(word.lower())

  return lower_list

def apply_stemming(word_list, flag=True):
  
  if flag:
    word_list = list(set(word_list))

  ps = PorterStemmer()  

  stemmed_words = []

  for word in word_list:
    stemmed_words.append(ps.stem(word))

  if flag:
    stemmed_words = list(set(stemmed_words))

  return stemmed_words

In [ ]:
def match_words(Minute_nouns, Transcript_nouns):

  Minute_nouns = list(set(Minute_nouns))  
  Transcript_nouns = list(set(Transcript_nouns))

  Count = 0
  for word in Minute_nouns:
    if word in Transcript_nouns:
      Count += 1
      

In [ ]:
To_remove_list = ['something', 'meeting', 'transcript', 'arrival', 'system', 'suggestion', 'annotation']

def remove_from(text_list):

  text_list = list(set(lowercase(text_list)))

  sample = []

  for word in text_list:
    if word not in To_remove_list:
      sample.append(word)

  return sample

In [ ]:
# Remove the content within square brackets 
minute = remove_cntsqbr(minutes1)
transcript = remove_cntsqbr(transcripts1)  

# Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
Minute_nouns = aggregate_nouns(stop_words, minute)
Transcript_nouns = aggregate_nouns(stop_words, transcript)

Minute_nouns = remove_from(Minute_nouns)
Transcript_nouns = remove_from(Transcript_nouns)

# Minute_nouns = apply_stemming(Minute_nouns)
# Transcript_nouns = apply_stemming(Transcript_nouns, False)

print(list(set(Minute_nouns)))

Minute_nouns = list(set(Minute_nouns))
Transcript_nouns = list(set(Transcript_nouns))
Count = 0
for word in Minute_nouns:
  if word in Transcript_nouns:
    # print(word)
    Count+=1
  else:
    pass
    print(word)

['person', 'source', 'combination', 'translation', 'speaker', 'language', 'combinaton']
combination
combinaton


In [ ]:
# Remove the content within square brackets 
minute = remove_cntsqbr(minutes2)
transcript = remove_cntsqbr(transcripts2)  

# Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
Minute_nouns = aggregate_nouns(stop_words, minute)
Transcript_nouns = aggregate_nouns(stop_words, transcript)


Minute_nouns = remove_from(Minute_nouns)
Transcript_nouns = remove_from(Transcript_nouns)

# Minute_nouns = apply_stemming(Minute_nouns)
# Transcript_nouns = apply_stemming(Transcript_nouns, False)

print(list(set(Minute_nouns)))

Minute_nouns = list(set(Minute_nouns))
Transcript_nouns = list(set(Transcript_nouns))
Count = 0
for word in Minute_nouns:
  if word in Transcript_nouns:
    # print(word)
    Count+=1
  else:
    pass
    print(word)

['lemmatization', 'constraint', 'implementation', 'payment', 'expression', 'architecture', 'baseline', 'document', 'difference', 'anything', 'achieve', 'research', 'lemmas', 'translation', 'quality', 'confidence', 'approach', 'versus', 'graphs', 'source', 'budget', 'estimation']
payment
expression


In [ ]:
# Remove the content within square brackets 
minute = remove_cntsqbr(minutes3)
transcript = remove_cntsqbr(transcripts3)  

# Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
Minute_nouns = aggregate_nouns(stop_words, minute)
Transcript_nouns = aggregate_nouns(stop_words, transcript)


Minute_nouns = remove_from(Minute_nouns)
Transcript_nouns = remove_from(Transcript_nouns)

# Minute_nouns = apply_stemming(Minute_nouns)
# Transcript_nouns = apply_stemming(Transcript_nouns, False)

print(list(set(Minute_nouns)))

Minute_nouns = list(set(Minute_nouns))
Transcript_nouns = list(set(Transcript_nouns))
Count = 0
for word in Minute_nouns:
  if word in Transcript_nouns:
    print(word)
    Count+=1
  else:
    pass
    # print(word)

print(Count, len(Minute_nouns))

['discussion', 'development', 'involvement', 'compromise', 'clarification', 'interest', 'advice', 'cluster', 'therefore', 'synchronisation', 'specialise', 'progress', 'moment', 'summarization', 'everything', 'relation', 'research', 'organisation', 'creation', 'repository', 'aggregate', 'release', 'literature', 'storage', 'workload', 'review', 'format', 'decision', 'agreement', 'information', 'privacy']
moment
everything
2 31


In [ ]:
# Remove the content within square brackets 
minute = remove_cntsqbr(minutes4)
transcript = remove_cntsqbr(transcripts4)  

# Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
Minute_nouns = aggregate_nouns(stop_words, minute)
Transcript_nouns = aggregate_nouns(stop_words, transcript)


Minute_nouns = remove_from(Minute_nouns)
Transcript_nouns = remove_from(Transcript_nouns)

# Minute_nouns = apply_stemming(Minute_nouns)
# Transcript_nouns = apply_stemming(Transcript_nouns, False)

print(list(set(Minute_nouns)))

Minute_nouns = list(set(Minute_nouns))
Transcript_nouns = list(set(Transcript_nouns))
Count = 0
for word in Minute_nouns:
  if word in Transcript_nouns:
    # print(word)
    Count+=1
  else:
    pass
    print(word)

['discussion', 'development', 'involvement', 'compromise', 'clarification', 'interest', 'advice', 'cluster', 'therefore', 'synchronisation', 'specialise', 'progress', 'moment', 'summarization', 'everything', 'relation', 'research', 'organisation', 'creation', 'repository', 'aggregate', 'release', 'literature', 'storage', 'workload', 'review', 'format', 'decision', 'agreement', 'information', 'privacy']
discussion
development
involvement
compromise
clarification
interest
advice
therefore
synchronisation
specialise
progress
moment
summarization
relation
research
organisation
creation
repository
aggregate
release
literature
storage
workload
review
decision
agreement
privacy


In [ ]:
# Remove the content within square brackets 
minute = remove_cntsqbr(minutes5)
transcript = remove_cntsqbr(transcripts5)  

# Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
Minute_nouns = aggregate_nouns(stop_words, minute)
Transcript_nouns = aggregate_nouns(stop_words, transcript)


Minute_nouns = remove_from(Minute_nouns)
Transcript_nouns = remove_from(Transcript_nouns)

# Minute_nouns = apply_stemming(Minute_nouns)
# Transcript_nouns = apply_stemming(Transcript_nouns, False)

print(list(set(Minute_nouns)))

Minute_nouns = list(set(Minute_nouns))
Transcript_nouns = list(set(Transcript_nouns))
Count = 0
for word in Minute_nouns:
  if word in Transcript_nouns:
    # print(word)
    Count+=1
  else:
    pass
    print(word)

['review', 'metaform', 'success', 'project', 'milestone', 'anything', 'dissemination']
metaform
success


### Back to Score 1


In [ ]:
def cnt_aggregator(minute, transcript, threshold = 0.8):

  minute_cnt = get_brac_cnt(minute)
  if len(minute_cnt) == 0:
    return 0
  else:
    trans_cnt = get_brac_cnt(transcript)
    match = chk_person_match(minute_cnt, trans_cnt)
    m_len = len(match)
    min_len = len(minute_cnt)
    score = len(match)/len(minute_cnt)
    if score > threshold:
      return 1
    else: 
      return 0

In [ ]:
score_1_results = []

for idx, val in enumerate(Labels):
  print(f'Iterating over file: {idx}')
  try:
    minutes = open(minutes_files[idx], 'r').readlines()
    transcript = open(transcript_files[idx], 'r').readlines()
    pred_label = cnt_aggregator(minutes, transcript)
    score_1_results.append(pred_label)
  except Exception as e:
    print('Ignoring this files')

### Check Confusion with different threshold

In [ ]:
Thresholds = [0.7, 0.75, 0.8, 0.85]
Scores = dict()

for thres in Thresholds:

  score_1_results = []

  print(f'Threshold: {thres}')
  for idx, val in enumerate(Labels):
    try:
      minutes = open(minutes_files[idx], 'r').readlines()
      transcript = open(transcript_files[idx], 'r').readlines()
      pred_label = cnt_aggregator(minutes, transcript, thres)
      score_1_results.append(pred_label)
    except Exception as e:
      print('Ignoring this files')

  Scores[f'thres_{thres}'] = score_1_results

Threshold: 0.7
Threshold: 0.75
Threshold: 0.8
Threshold: 0.85


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
Scores.keys()

dict_keys(['thres_0.7', 'thres_0.75', 'thres_0.8', 'thres_0.85'])

In [ ]:
cf_val = {
    'Threshold': [],
    'tn': [],
    'fp': [],
    'fn': [],
    'tp': []
}

for thres in Thresholds:

  cf_val['Threshold'].append(thres)

  y_pred = Scores[f'thres_{thres}']
  y_true = Labels
  tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
  cf_val['tn'].append(tn)
  cf_val['fp'].append(fp)
  cf_val['fn'].append(fn)
  cf_val['tp'].append(tp)


cf_val

{'Threshold': [0.7, 0.75, 0.8, 0.85],
 'fn': [34, 40, 44, 46],
 'fp': [323, 283, 245, 217],
 'tn': [405, 445, 483, 511],
 'tp': [81, 75, 71, 69]}

## Calculate Score 2

In [ ]:
# get the nouns list for both minutes and Transcript
# apply aggregate function

def overall_aggregator(minute, transcript, stopwords):

  # Remove the content within square brackets 
  minute = remove_cntsqbr(minute)
  transcript = remove_cntsqbr(transcript)  

  # Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
  Minute_nouns = aggregate_nouns(stopwords, minute)
  Transcript_nouns = aggregate_nouns(stopwords, transcript)

  # Apply Counter to Transcripts
  Transcript_nouns_Counter = Counter(Transcript_nouns)

  # check the word count corresponding to similarity of minute nouns in transcript nouns
  word_count = meet_word_count(Minute_nouns, Transcript_nouns_Counter)

  # Calculat Average distinct word count of minutes nouns in transcript nouns
  avg_value = calculate_Avg(word_count)

  return avg_value

average = overall_aggregator(minutes, transcripts, stop_words)

overall_aggregator(minutes, transcripts, stop_words)

3.857142925262451

In [ ]:
import os

root_dir = '/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/'
train_dir = os.path.join(root_dir, 'train/')
dev_dir = os.path.join(root_dir, 'dev/')

In [ ]:
score_2_results = []

for idx, val in enumerate(Labels):
  print(f'Iterating over file: {idx}')
  try:
    minutes = open(minutes_files[idx], 'r').readlines()
    transcript = open(transcript_files[idx], 'r').readlines()
    avg_val = overall_aggregator(minute=minutes, transcript=transcript, stopwords=stop_words)
    score_2_results.append(avg_val)
  except Exception as e:
    print('Ignoring this files')

## Calculate Score 3

### Defining Scores

In [ ]:
from nltk.tokenize import word_tokenize

def cosine_similarity(X,Y):
    
    X_list = word_tokenize(X) 
    Y_list = word_tokenize(Y)
    sw = stopwords.words('english') 
    l1 =[];l2 =[]
    X_set = {w for w in X_list if not w in sw} 
    Y_set = {w for w in Y_list if not w in sw}


    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0


    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    return cosine

In [ ]:
from difflib import SequenceMatcher

def sequence_matcher(a, b):
  return SequenceMatcher(None, a, b).ratio()

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

def Rouge_Score(hypothesis, reference):
  rouge = Rouge()
  scores = rouge.get_scores(hypothesis, reference)
  
  return scores

In [ ]:
def res(a,b):
    return len(set(a.split(' ')) & set(b.split(' '))) / float(len(set(a.split(' ')) | set(b.split(' ')))) *100

In [ ]:
def aggregate_concat(minutes, transcripts):
 
  # Remove the content within square brackets 
  minute = remove_cntsqbr(minutes)
  transcript = remove_cntsqbr(transcripts)  
 
  # Remove punctutations, stop_words -> Add pos_tag -> Get NNS || NN
  Minute_nouns = aggregate_nouns(stop_words, minute)
  Transcript_nouns = aggregate_nouns(stop_words, transcript)
 
  minute_str, transcript_str = concat_nouns(Minute_nouns, Transcript_nouns)
 
  cos = cosine_similarity(minute_str, transcript_str)
  r_1 = Rouge_Score(minute_str, transcript_str)[0]['rouge-1']['f']
  r_l = Rouge_Score(minute_str, transcript_str)[0]['rouge-l']['f']
  seq = sequence_matcher(minute_str, transcript_str)
  reS = res(minute_str, transcript_str)
 
  return cos, r_1, r_l, seq, reS

In [ ]:
cos, r_1, r_l, seq, reS = aggregate_concat(minutes, transcripts)
cos, r_1, r_l, seq, reS

(0.0, 0.0, 0.0, 0.009735744089012517, 0.0)

In [ ]:
print(len(Score_a))

563


### Implementation

In [ ]:
import os
 
root_dir = '/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/'
train_dir = os.path.join(root_dir, 'train/')
dev_dir = os.path.join(root_dir, 'dev/')

In [ ]:
Score_a, Score_b, Score_c, Score_d, Score_e = [], [], [], [], []

for idx, val in enumerate(Labels):
  print(f'Iterating over file: {idx}')
  try:
    minutes = open(minutes_files[idx], 'r').readlines()
    transcript = open(transcript_files[idx], 'r').readlines()
    cos, r_1, r_l, seq, reS = aggregate_concat(minutes, transcript)
    Score_a.append(cos)
    Score_b.append(r_1)
    Score_c.append(r_l)
    Score_d.append(seq)
    Score_e.append(reS)

  except Exception as e:
    print(e)
    print('Ignoring this files')

In [ ]:
len(Score_e), len(Score_a), len(Score_b), len(Score_c), len(Score_d), len(Labels), len(score_1_results), len(score_2_results)

(843, 843, 843, 843, 843, 843, 843, 843)

In [ ]:
import pandas as pd

Score_dataframe = pd.DataFrame({
    'Score_1': score_1_results,
    'Score_2': score_2_results,
    'Cosine_Similarity': Score_a,
    'Rouge_1': Score_b,
    'Rouge_l' : Score_c,
    'Sequence_matcher' : Score_d,
    'Res' : Score_e,
    'Labels': Labels,
})

In [ ]:
Score_dataframe

,Score_1,Score_2,Cosine_Similarity,Rouge_1,Rouge_l,Sequence_matcher,Res,Labels
0,1,3.000000,0.143019,0.070175,0.023392,0.008951,3.636364,True
1,0,4.200000,0.227284,0.142857,0.079365,0.007048,7.692308,True
2,0,3.777778,0.139686,0.056075,0.028037,0.013151,2.884615,True
3,1,1.888889,0.197674,0.164706,0.070588,0.018592,8.974359,True
4,1,4.000000,0.340087,0.241071,0.080357,0.014851,13.705584,True
...,...,...,...,...,...,...,...,...
838,1,3.625000,0.181362,0.142857,0.053571,0.017587,7.692308,False
839,0,1.800000,0.051640,0.029197,0.014599,0.019370,1.481481,False
840,0,2.000000,0.071429,0.020000,0.020000,0.009060,1.010101,False
841,0,2.000000,0.062568,0.042857,0.042857,0.012589,2.189781,False


In [ ]:
%cd /content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/

/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en


In [ ]:
%cd /content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/

/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en


In [ ]:
Score_dataframe.to_csv('task-B-Scores.csv')

## Create Dataframe


In [ ]:
import pandas as pd

Score_dataframe = pd.DataFrame({
    'Score_1': score_1_results,
    'Score_2': score_2_results,
    'Labels' : Labels
})

In [ ]:
# format Score_1, Score_2, Labels
Score_dataframe

,Score_1,Score_2,Labels
0,1,2.000000,True
1,0,2.066667,True
2,0,2.600000,True
3,1,2.733333,True
4,1,2.648649,True
...,...,...,...
838,0,2.500000,False
839,0,1.833333,False
840,0,3.000000,False
841,0,2.750000,False


In [ ]:
Score_1 = np.array(Score_dataframe[Score_dataframe['Labels'] == True]['Score_1'])

In [ ]:
%cd /content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/

/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en


In [ ]:
Score_dataframe.to_csv('task-B-Scores.csv')

## Classification on the task B Scores.csv

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
%cd /content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en/

/content/drive/MyDrive/automin-2021-confindential-data-main/task-B-en


In [ ]:
task_B_data = pd.read_csv('task-B-Scores.csv')

In [ ]:
task_B_data = task_B_data.drop('Unnamed: 0', axis=1)

In [ ]:
Data = task_B_data
Data

,Score_1,Score_2,Cosine_Similarity,Rouge_1,Rouge_l,Sequence_matcher,Res,Labels
0,1,3.000000,0.143019,0.070175,0.023392,0.008951,3.636364,True
1,0,4.200000,0.227284,0.142857,0.079365,0.007048,7.692308,True
2,0,3.777778,0.139686,0.056075,0.028037,0.013151,2.884615,True
3,1,1.888889,0.197674,0.164706,0.070588,0.018592,8.974359,True
4,1,4.000000,0.340087,0.241071,0.080357,0.014851,13.705584,True
...,...,...,...,...,...,...,...,...
838,1,3.625000,0.181362,0.142857,0.053571,0.017587,7.692308,False
839,0,1.800000,0.051640,0.029197,0.014599,0.019370,1.481481,False
840,0,2.000000,0.071429,0.020000,0.020000,0.009060,1.010101,False
841,0,2.000000,0.062568,0.042857,0.042857,0.012589,2.189781,False


In [ ]:
True_Labels = [label for label in Data['Labels'] if label ==  True]
False_Labels =  [label for label in Data['Labels'] if label ==  False]
len(True_Labels)

115

In [ ]:
task_B_data_Actual = pd.read_csv('task-B-Scores.csv')

In [ ]:
Data['Score_2'] = Data['Score_2'].fillna(0)

In [ ]:
Data.head

<bound method NDFrame.head of      Score_1   Score_2  Cosine_Similarity  ...  Sequence_matcher        Res  Labels
0          1  3.000000           0.143019  ...          0.008951   3.636364    True
1          0  4.200000           0.227284  ...          0.007048   7.692308    True
2          0  3.777778           0.139686  ...          0.013151   2.884615    True
3          1  1.888889           0.197674  ...          0.018592   8.974359    True
4          1  4.000000           0.340087  ...          0.014851  13.705584    True
..       ...       ...                ...  ...               ...        ...     ...
838        1  3.625000           0.181362  ...          0.017587   7.692308   False
839        0  1.800000           0.051640  ...          0.019370   1.481481   False
840        0  2.000000           0.071429  ...          0.009060   1.010101   False
841        0  2.000000           0.062568  ...          0.012589   2.189781   False
842        1  4.666667           0.044065  ...

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Data.drop(['Labels', 'Score_1'], axis=1), Data['Labels'], test_size=0.33, random_state=0)

### SVC 

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel="rbf", gamma='auto')
clf.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
clf.score(X_test, y_test), clf.score(X_train, y_train)

(0.8888888888888888, 0.9095744680851063)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
confusion_matrix(y_test, pred)

array([[235,   4],
       [ 27,  13]])

###  Random Forst

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Classifier = RandomForestClassifier(max_depth=200, random_state=42)
Classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=200, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = Classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9175627240143369

In [ ]:
confusion_matrix(y_test, y_pred)

array([[234,   5],
       [ 18,  22]])

### Imbalance Classifier

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SVMSMOTE

In [ ]:
sm = SVMSMOTE(random_state=42)

X_res, y_res = sm.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf', gamma = 'auto')
clf.fit(X_res,y_res)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
print(clf.score(X_test, y_test))

pred = clf.predict(X_test)

0.8207885304659498


In [ ]:
confusion_matrix(y_test, pred)

array([[204,  35],
       [ 15,  25]])

## Oversample true data

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_Train, y_Train = ros.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_Train.shape

(978, 6)

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma= 'auto')
clf.fit(X_Train,y_Train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
clf.score(X_test, y_test), clf.score(X_Train, y_Train)

(0.8279569892473119, 0.8149284253578732)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Classifier = RandomForestClassifier(max_depth=200)
Classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=200, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# X_T, X_t, y_T, y_t = train_test_split(X_Train, y_Train, test_size=0.3, random_state=0)

In [ ]:
Classifier.score(X_test, y_test), Classifier.score(X_train, y_train)

(0.921146953405018, 1.0)

In [ ]:
y_pred = Classifier.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[234,   5],
       [ 17,  23]])

In [ ]:
X_Train[y_Train==True].shape, X_Train[y_Train==False].shape

((489, 7), (489, 7))

In [ ]:
X_train[y_train==True].shape, X_train[y_train==False].shape

((75, 7), (489, 7))

In [ ]:
y_pred = Classifier.predict(X_Train)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_Train, y_pred).ravel()
tn, fp, fn, tp

(489, 0, 0, 489)

### GDA

In [ ]:
from sklearn.naive_bayes import GaussianNB

CLF = GaussianNB()
CLF.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y_Pred = CLF.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_Pred)

0.8494623655913979

In [ ]:
confusion_matrix(y_test, y_Pred)

array([[214,  25],
       [ 17,  23]])